# Case Study: Clara

![Logo](https://github.com/Nacho-o/public-data/blob/main/image.png?raw=true)

#### Import Required Libraries and Load Datasets

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

In [3]:
url_home_page = 'https://raw.githubusercontent.com/Nacho-o/public-data/main/data/home_page_table.csv'
url_search_page = 'https://raw.githubusercontent.com/Nacho-o/public-data/main/data/search_page_table.csv'
url_payment_page = 'https://raw.githubusercontent.com/Nacho-o/public-data/main/data/payment_page_table.csv'
url_payment_confirmation = 'https://raw.githubusercontent.com/Nacho-o/public-data/main/data/payment_confirmation_table.csv'
url_user_table = 'https://raw.githubusercontent.com/Nacho-o/public-data/main/data/user_table.csv'

In [6]:
home_page = pd.read_csv(url_home_page)
search_page = pd.read_csv(url_search_page)
payment_page = pd.read_csv(url_payment_page)
payment_confirmation = pd.read_csv(url_payment_confirmation)
user_table = pd.read_csv(url_user_table)

### User Segmentation Based on Registration Date

Given that the **maximum date** in the dataset is **April 30, 2015**, we assume this is the "current day" for the analysis. To segment users into "new" and "old", we use a threshold of **30 days** prior to the maximum date. Users registered on or after **March 31, 2015** are considered "new", while those registered before this date are considered "old". 

This segmentation helps us analyze differences in behavior between newly registered users and those with longer histories on the platform.

In [7]:
user_table['date'] = pd.to_datetime(user_table['date'], errors='coerce')
max_date = user_table['date'].max()
min_date = user_table['date'].min()

print(max_date)
print(min_date)

2015-04-30 00:00:00
2015-01-01 00:00:00


In [4]:
threshold_days = 30
threshold_date = max_date - pd.Timedelta(days=threshold_days)
user_table['user_type'] = user_table['date'].apply(lambda x: 'new' if x >= threshold_date else 'old')

In [220]:
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [221]:
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [222]:
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [223]:
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


In [224]:
payment_confirmation.head()

,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


## Exploratory Data Analysis (EDA)

#### Merge Datasets for Analysis

In [10]:
merged_data = user_table.merge(home_page, on='user_id', how='left', suffixes=('', '_home')) \
                        .merge(search_page, on='user_id', how='left', suffixes=('', '_search')) \
                        .merge(payment_page, on='user_id', how='left', suffixes=('', '_payment')) \
                        .merge(payment_confirmation, on='user_id', how='left', suffixes=('', '_confirmation'))
merged_data

,user_id,date,device,sex,user_type,page,page_search,page_payment,page_confirmation
0,450007,2015-02-28,Desktop,Female,old,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,old,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,new,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,old,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,old,home_page,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
90395,307667,2015-03-30,Desktop,Female,old,home_page,NaN,NaN,NaN
90396,642989,2015-02-08,Desktop,Female,old,home_page,search_page,NaN,NaN
90397,659645,2015-04-13,Desktop,Male,new,home_page,search_page,NaN,NaN
90398,359779,2015-03-23,Desktop,Male,old,home_page,NaN,NaN,NaN


In [207]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90400 entries, 0 to 90399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            90400 non-null  int64         
 1   date               90400 non-null  datetime64[ns]
 2   device             90400 non-null  object        
 3   sex                90400 non-null  object        
 4   user_type          90400 non-null  object        
 5   page               90400 non-null  object        
 6   page_search        45200 non-null  object        
 7   page_payment       6030 non-null   object        
 8   page_confirmation  452 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 6.2+ MB


In [11]:
merged_data['date'] = pd.to_datetime(merged_data['date'], errors='coerce')

In [12]:
user_test_data = merged_data.loc[merged_data['user_id'] == 123100]
user_test_data

,user_id,date,device,sex,user_type,page,page_search,page_payment,page_confirmation
18329,123100,2015-02-24,Desktop,Female,old,home_page,search_page,payment_page,payment_confirmation_page


#### Conversion Rates Analysis for All Users

In [13]:
total_home = merged_data['page'].notna().sum()
total_search = merged_data['page_search'].notna().sum()
total_payment = merged_data['page_payment'].notna().sum()
total_confirmation = merged_data['page_confirmation'].notna().sum()

In [196]:
labels = ['Home', 'Search', 'Payment', 'Confirmation']
total_users = [
    total_home,
    total_search,
    total_payment,
    total_confirmation
]

In [209]:
conversion_to_home = total_home / total_home 
conversion_to_search = total_search / total_home
conversion_to_payment = total_payment / total_search
conversion_to_confirmation = total_confirmation / total_payment

conversion_rates_all = [
    conversion_to_home * 100,
    conversion_to_search * 100,
    conversion_to_payment * 100,
    conversion_to_confirmation * 100
]

overall_conversion_rate = (
    total_confirmation / total_home * 100
)

In [210]:
df = pd.DataFrame(dict(
    stage=labels,
    percentage=conversion_rates_all,
    number= total_users
))

fig = px.funnel(df, x='percentage', y='stage', text='number',
                title='Conversion Funnel from Home to Confirmation')

fig.update_layout(width=900)
fig.update_traces(texttemplate='%{text:,} users<br>%{x:.2f}%', textposition='inside')

fig.add_annotation(
    text=f"Overall Conversion Rate: {overall_conversion_rate:.2f}%",
    xref="paper", yref="paper",
    x=0.5, y=-0.2,
    showarrow=False,
    font=dict(size=12, color="black")
)

fig.show()

#### Conversion Rates for New and Old Users

In [18]:
total_new_users_home = merged_data[merged_data['user_type'] == 'new']['page'].notna().sum()
total_old_users_home = merged_data[merged_data['user_type'] == 'old']['page'].notna().sum()

total_new_users_search = merged_data[merged_data['user_type'] == 'new']['page_search'].notna().sum()
total_old_users_search = merged_data[merged_data['user_type'] == 'old']['page_search'].notna().sum()

total_new_users_payment = merged_data[merged_data['user_type'] == 'new']['page_payment'].notna().sum()
total_old_users_payment = merged_data[merged_data['user_type'] == 'old']['page_payment'].notna().sum()

total_new_users_confirmation = merged_data[merged_data['user_type'] == 'new']['page_confirmation'].notna().sum()
total_old_users_confirmation = merged_data[merged_data['user_type'] == 'old']['page_confirmation'].notna().sum()

In [19]:
total_new_users = [
    total_new_users_home,
    total_new_users_search,
    total_new_users_payment,
    total_new_users_confirmation,
]

total_old_users = [
    total_old_users_home,
    total_old_users_search,
    total_old_users_payment,
    total_old_users_confirmation,
]

In [176]:
conversion_rate_new = [
    total_new_users_home / total_new_users_home * 100,
    total_new_users_search / total_new_users_home * 100,
    total_new_users_payment / total_new_users_search * 100,
    total_new_users_confirmation / total_new_users_payment * 100,
]

overall_conversion_rate_new = (
    total_new_users_confirmation / total_new_users_home * 100
)

conversion_rate_old = [
    total_old_users_home / total_old_users_home * 100,
    total_old_users_search / total_old_users_home * 100,
    total_old_users_payment / total_old_users_search * 100,
    total_old_users_confirmation / total_old_users_payment * 100,
]


overall_conversion_rate_old = (
    total_old_users_confirmation / total_old_users_home * 100
)

In [179]:
conversion_data = pd.DataFrame({
    'stage': labels * 2,
    'percentage': conversion_rate_old + conversion_rate_new,
    'number': total_old_users + total_new_users,
    'users': ['Old'] * 4 + ['New'] * 4
})

conversion_data['text'] = conversion_data.apply(
    lambda row: f"{row['number']:,} users<br>{row['percentage']:.2f}%", axis=1
)

fig = px.funnel(conversion_data, x='percentage', y='stage', text='text', 
                color='users',
                title='Conversion Rates for Old and New Users')

fig.update_traces(
    texttemplate='%{text}'
)

fig.update_layout(
    width=1100, 
    height=800,
    legend_title_text='Users',
    legend=dict(
        x=0.8,
        y=1.0,
        traceorder='normal',
        font=dict(size=12)
    )
)

fig.add_annotation(
    text=f"Overall Conversion Rate (Old Users): {overall_conversion_rate_old:.2f}%",
    xref="paper", yref="paper",
    x=0.15, y=-0.1,
    showarrow=False,
    font=dict(size=12, color="black")
)

fig.add_annotation(
    text=f"Overall Conversion Rate (New Users): {overall_conversion_rate_new:.2f}%",
    xref="paper", yref="paper",
    x=0.85, y=-0.1,
    showarrow=False,
    font=dict(size=12, color="black")
)

fig.show()

#### Conversion Rates for New Users by Device

In [80]:
# New device users
total_new_mobile_home = merged_data[(merged_data['user_type'] == 'new') & (merged_data['device'] == 'Mobile')]['page'].notna().sum()
total_new_mobile_search = merged_data[(merged_data['user_type'] == 'new') & (merged_data['device'] == 'Mobile')]['page_search'].notna().sum()
total_new_mobile_payment = merged_data[(merged_data['user_type'] == 'new') & (merged_data['device'] == 'Mobile')]['page_payment'].notna().sum()
total_new_mobile_confirmation = merged_data[(merged_data['user_type'] == 'new') & (merged_data['device'] == 'Mobile')]['page_confirmation'].notna().sum()

# New desktop users
total_new_desktop_home = merged_data[(merged_data['user_type'] == 'new') & (merged_data['device'] == 'Desktop')]['page'].notna().sum()
total_new_desktop_search = merged_data[(merged_data['user_type'] == 'new') & (merged_data['device'] == 'Desktop')]['page_search'].notna().sum()
total_new_desktop_payment = merged_data[(merged_data['user_type'] == 'new') & (merged_data['device'] == 'Desktop')]['page_payment'].notna().sum()
total_new_desktop_confirmation = merged_data[(merged_data['user_type'] == 'new') & (merged_data['device'] == 'Desktop')]['page_confirmation'].notna().sum()

total_new_mobile_users = [total_new_mobile_home, total_new_mobile_search, total_new_mobile_payment, total_new_mobile_confirmation]
total_new_desktop_users = [total_new_desktop_home, total_new_desktop_search, total_new_desktop_payment, total_new_desktop_confirmation]

In [180]:
conversion_rate_new_mobile = [
    (total_new_mobile_home/total_new_mobile_home) * 100,
    (total_new_mobile_search / total_new_mobile_home) * 100 if total_new_mobile_home > 0 else 0,
    (total_new_mobile_payment / total_new_mobile_search) * 100 if total_new_mobile_search > 0 else 0,
    (total_new_mobile_confirmation / total_new_mobile_payment) * 100 if total_new_mobile_payment > 0 else 0
]

overall_conversion_rate_new_mobile = (
    total_new_mobile_confirmation / total_new_mobile_home * 100
)

conversion_rate_new_desktop = [
    (total_new_desktop_home/total_new_desktop_home) * 100,
    (total_new_desktop_search / total_new_desktop_home) * 100 if total_new_desktop_home > 0 else 0,
    (total_new_desktop_payment / total_new_desktop_search) * 100 if total_new_desktop_search > 0 else 0,
    (total_new_desktop_confirmation / total_new_desktop_payment) * 100 if total_new_desktop_payment > 0 else 0
]

overall_conversion_rate_new_desktop = (
    total_new_desktop_confirmation / total_new_desktop_home * 100
)

In [181]:
conversion_data = pd.DataFrame({
    'stage': labels * 2,
    'percentage': conversion_rate_new_mobile + conversion_rate_new_desktop,
    'number': total_new_mobile_users + total_new_desktop_users,
    'device': ['Mobile'] * 4 + ['Desktop'] * 4
})

conversion_data['text'] = conversion_data.apply(
    lambda row: f"{row['number']:,} users<br>{row['percentage']:.2f}%", axis=1
)

fig = px.funnel(conversion_data, x='percentage', y='stage', text='text', 
                color='device',
                title='Conversion Funnel for New Users (Mobile vs Desktop)')

fig.update_traces(
    texttemplate='%{text}'
)

fig.update_layout(
    width=1100, 
    height=800,
    legend_title_text='Device',
    legend=dict(
        x=0.8,
        y=1.0,
        traceorder='normal',
        font=dict(size=12)
    )
)

fig.add_annotation(
    text=f"Overall Conversion Rate (Mobile Users): {overall_conversion_rate_new_mobile:.2f}%",
    xref="paper", yref="paper",
    x=0.15, y=-0.1,
    showarrow=False,
    font=dict(size=12, color="black")
)

fig.add_annotation(
    text=f"Overall Conversion Rate (Desktop Users): {overall_conversion_rate_new_desktop:.2f}%",
    xref="paper", yref="paper",
    x=0.85, y=-0.1,
    showarrow=False,
    font=dict(size=12, color="black")
)

fig.show()

#### Conversion Rates for New Users by Sex

In [200]:
total_new_female_home = merged_data[(merged_data['user_type'] == 'new') & (merged_data['sex'] == 'Female')]['page'].notna().sum()
total_new_female_search = merged_data[(merged_data['user_type'] == 'new') & (merged_data['sex'] == 'Female')]['page_search'].notna().sum()
total_new_female_payment = merged_data[(merged_data['user_type'] == 'new') & (merged_data['sex'] == 'Female')]['page_payment'].notna().sum()
total_new_female_confirmation = merged_data[(merged_data['user_type'] == 'new') & (merged_data['sex'] == 'Female')]['page_confirmation'].notna().sum()

total_new_male_home = merged_data[(merged_data['user_type'] == 'new') & (merged_data['sex'] == 'Male')]['page'].notna().sum()
total_new_male_search = merged_data[(merged_data['user_type'] == 'new') & (merged_data['sex'] == 'Male')]['page_search'].notna().sum()
total_new_male_payment = merged_data[(merged_data['user_type'] == 'new') & (merged_data['sex'] == 'Male')]['page_payment'].notna().sum()
total_new_male_confirmation = merged_data[(merged_data['user_type'] == 'new') & (merged_data['sex'] == 'Male')]['page_confirmation'].notna().sum()

total_new_female_users = [total_new_female_home, total_new_female_search, total_new_female_payment, total_new_female_confirmation]
total_new_male_users = [total_new_male_home, total_new_male_search, total_new_male_payment, total_new_male_confirmation]

In [201]:
conversion_rate_new_female = [
    (total_new_female_home / total_new_female_home) * 100,
    (total_new_female_search / total_new_female_home) * 100 if total_new_female_home > 0 else 0,
    (total_new_female_payment / total_new_female_search) * 100 if total_new_female_search > 0 else 0,
    (total_new_female_confirmation / total_new_female_payment) * 100 if total_new_female_payment > 0 else 0
]

overall_conversion_rate_new_female = (
    total_new_female_confirmation / total_new_female_home * 100
)

conversion_rate_new_male = [
    (total_new_male_home / total_new_male_home) * 100,
    (total_new_male_search / total_new_male_home) * 100 if total_new_male_home > 0 else 0,
    (total_new_male_payment / total_new_male_search) * 100 if total_new_male_search > 0 else 0,
    (total_new_male_confirmation / total_new_male_payment) * 100 if total_new_male_payment > 0 else 0
]

overall_conversion_rate_new_male = (
    total_new_male_confirmation / total_new_male_home * 100
)

In [203]:
conversion_data = pd.DataFrame({
    'stage': labels * 2,
    'percentage': conversion_rate_new_female + conversion_rate_new_male,
    'number': total_new_female_users + total_new_male_users,
    'sex': ['Female'] * 4 + ['Male'] * 4
})

conversion_data['text'] = conversion_data.apply(
    lambda row: f"{row['number']:,} users<br>{row['percentage']:.2f}%", axis=1
)

fig = px.funnel(conversion_data, x='percentage', y='stage', text='text', 
                color='sex',
                title='Conversion Funnel for New Users (Female vs Male)')

fig.update_traces(
    texttemplate='%{text}'
)

fig.update_layout(
    width=1100, 
    height=700,
    legend_title_text='Sex',
    legend=dict(
        x=0.8,
        y=1.0,
        traceorder='normal',
        font=dict(size=12)
    )
)

fig.add_annotation(
    text=f"Overall Conversion Rate (Female Users): {overall_conversion_rate_new_female:.2f}%",
    xref="paper", yref="paper",
    x=0.15, y=-0.1,
    showarrow=False,
    font=dict(size=12, color="black")
)

fig.add_annotation(
    text=f"Overall Conversion Rate (Male Users): {overall_conversion_rate_new_male:.2f}%",
    xref="paper", yref="paper",
    x=0.85, y=-0.1,
    showarrow=False,
    font=dict(size=12, color="black")
)

fig.show()

# Conversion Funnel Analysis for E-commerce Website

## Introduction
This analysis examines the conversion rates at different stages of the funnel on your e-commerce platform, focusing on the difference between **new users** and **old users**. The goal is to provide insights into where the conversion process breaks down and suggest strategies to improve the sales pipeline, especially for new users.

## Conversion Rates Breakdown

### Overall Conversion Rates (Old vs New Users):
- **Home to Search**:
  - Old Users: 53.44%
  - New Users: 40.11%
- **Search to Payment**:
  - Old Users: 15.08%
  - New Users: 6.67%
- **Payment to Confirmation**:
  - Old Users: 7.47%
  - New Users: 7.69%
- **Overall Conversion Rate**:
  - Old Users: 0.6%
  - New Users: 0.21%

### New Users Conversion Rates by Device:
- **Home to Search**:
  - Desktop: 50.20% (**7,801 out of 15,541**)
  - Mobile: 19.98% (**1,555 out of 7,782**)
- **Search to Payment**:
  - Desktop: 3.97% (**310 out of 7,801**)
  - Mobile: 20.19% (**314 out of 1,555**)
- **Payment to Confirmation**:
  - Desktop: 7.42% (**23 out of 310**)
  - Mobile: 7.96% (**25 out of 314**)

## Key Insights

### 1. Significant Drop from Home to Search
- **Old vs New Users**: The conversion rate from **Home to Search** is **53.44%** for **old users** compared to **40.11%** for **new users**. This suggests that **new users** are struggling more with the **Home Page** and might not find it as intuitive or engaging as **old users**.
- **Device Impact (New Users)**: Among **new users**, **desktop** users perform much better (**50.20%**) compared to **mobile** users (**19.98%**), with **6,227 mobile users** dropping off at this stage. This highlights a significant issue with the **mobile** experience on the **Home Page**.

### 2. Severe Drop from Search to Payment for New Users
- **Old vs New Users**: The drop-off from **Search to Payment** is particularly large for **new users** at **6.67%**, compared to **15.08%** for **old users**. This suggests that **new users** struggle to engage with the product selection or search results, causing them to abandon the process.
- **Device Impact (New Users)**: On **desktop**, the conversion rate from **Search to Payment** for **new users** is only **3.97%**, with a loss of **7,491 users** at this stage. In contrast, **mobile** users have a much higher conversion rate of **20.19%**, indicating that **desktop** search functionality may need more immediate attention.

### 3. Stable Conversion from Payment to Confirmation
- **Old vs New Users**: The conversion rate from **Payment to Confirmation** is relatively stable across both **new users** (**7.69%**) and **old users** (**7.47%**), indicating that the payment process itself is functioning reasonably well for both groups.
- **Device Impact (New Users)**: There is also no significant difference between **desktop** and **mobile** in this stage, with both devices converting at approximately **7-8%**.

## Recommendations for Improvement

### 1. Improve the Home Page to Increase Engagement
- The drop-off from **Home to Search**, particularly for **new users** and **mobile users**, indicates a need to optimize the homepage experience:
  - **Mobile Optimization**: Ensure the homepage is mobile-friendly with faster load times, simplified navigation, and clear CTAs (call-to-actions).
  - **First-time User Incentives**: Offer promotions or discounts specifically for new users directly on the homepage to encourage them to perform a search.
  - **Simplified Navigation**: Ensure that search functionality is easily accessible and visually prominent on the **Home Page**, especially for mobile users.

### 2. Improve Search Experience to Boost Payment Conversions, Especially for Desktop Users
- The severe drop-off between **Search and Payment** for **new users** (especially on **desktop**) suggests issues with how products are displayed or searched:
  - **Optimize Search Relevance**: Improve search algorithms to ensure users are presented with relevant products based on their queries.
  - **Trust Signals and User Reviews**: Display customer reviews, product ratings, and badges (e.g., "best-seller") to boost user confidence in the products shown in the search results.
  - **Retarget Abandoned Searches**: Capture user data from searches and retarget users who leave without purchasing through personalized emails or ads. Offering limited-time discounts or promotions can further incentivize the purchase.
  - **Wishlist Functionality**: Allow users to save products to a wishlist and notify them if the price drops or if there is a sale on that item, offering an easy way to convert searches into future purchases.
  - **Analyze User Behavior**: Use tools like **heatmaps** or **session recordings** to analyze how users, particularly mobile users, interact with the search and payment pages, and identify pain points or friction that may be preventing them from moving further along the funnel.

### 3. Optimize the Payment Stage to Maximize Conversions
- Although the conversion rate from **Payment to Confirmation** is stable, there are still opportunities to capture more conversions:
  - **Expand Payment Methods**: Offer a broader variety of payment methods, such as digital wallets, local bank transfers, and installment options to cater to different user preferences.
  - **Transparent Pricing**: Ensure there are no hidden fees or unexpected costs that could discourage users from completing their purchase at checkout.
  - **Exit-Intent Pop-Ups**: Use exit-intent pop-ups offering discounts or free shipping to users who attempt to leave the **Payment Page** without completing their transaction.

## Conclusion

The **conversion funnel** for your e-commerce site shows key areas for improvement:
- **New users** struggle more than **old users**, particularly in the **Home to Search** and **Search to Payment** stages. But should You Be Concerned About the Drop from Home to Search?
  - **Home to Search Drop**: **7,740 users** are lost between **Home and Search** (about 50% of users). While this seems significant, this kind of drop is generally expected in the early stages of an e-commerce funnel. Many visitors land on the home page with no clear buying intent. Although this drop-off is somewhat normal, improving even a small percentage of retention here could have a substantial impact. Given the large number of users that leave at this stage, capturing even a fraction of them would result in a significant increase in the number of users progressing to the **Search** page and beyond.
  - **Search to Payment Drop**: However, losing **96%** of users between **Search and Payment** is alarming. This suggests deeper problems with the search experience, product listings, or the purchasing experience.
- **Mobile users** exhibit higher drop-offs at the **Home Page**, while **desktop users** face significant issues at the **Search to Payment** stage. Addressing these specific device-related challenges will improve the experience for **new users**.
- The **Payment to Confirmation** stage is relatively stable, but further enhancements to payment methods and pricing transparency could help increase conversions.
- It’s also worth investigating if there were any recent **deployments or updates** to the platform that could have inadvertently affected the user experience. For example, changes in the search functionality, loading times, or user interface design might have contributed to this drop in conversions, particularly if these changes were rolled out within the last month.
- Lastly, to provide better insights, it would be beneficial to analyze the sources from which users arrive at the home page to identify any patterns in lead origins and their subsequent purchasing behavior. Investigating where users go if they do not click on search, as well as which payment options are most appealing to them, could also yield valuable information for improving conversion rates.
- No significant differences were observed between **female** and **male** users in the funnel stages. The analysis by sex was not included in depth because the conversion rates for both groups were nearly identical.

By focusing on these key stages, especially for **new users**, and refining the mobile and desktop experiences, you can significantly improve overall conversion rates and sales volume.